# SCC0270 - Redes Neurais e Aprendizado Profundo
### Aula 7 - Prática (Redes neurais convolutivas)
**Daniel Penna Chaves Bertazzo - 10349561**

# Exercício 1
A função de ativação *softmax* é dada pela fórmula
<br><br>
$$ S(y_i) = \frac{e^{y_i}} {\sum_{j=1}^{K} e^{y_j}} $$
<br>
onde $K$ é o número de classes presentes no *dataset*. Tal função de ativação possui como entrada um vetor de $K$ elementos ($\in \mathbb{R}$) e o normaliza para uma distribuição de probabilidade (outro vetor com $K$ elementos, onde cada elemento representa uma probabilidade proporcional aos exponenciais dos valores de entrada). No contexto de redes neurais, é normalmente utilizada na camada de saída, pois pega como entrada os valores gerados pelas ativações dos neurônios da penúltima camada e os transforma em um vetor onde cada $S(y_i)$ representa a probabilidade da classe $i$ ser a predição correta para aquela instância do *dataset*. Por exemplo:
<br><br>
Suponha que possuímos um *dataset* com 3 classes diferentes e uma rede neural treinada para prever corretamente à qual classe uma dada instância pertence. Ao fornecer à rede tal exemplo, suponha que ela retorne como resultado o vetor:
<br><br>
$$ \begin{bmatrix} {0.02 \\
                    0.98 \\
                    0.00} \end{bmatrix} $$
<br>
onde cada linha representa uma classe $y$. Nesse caso, temos que
* $P(y=1) = 2\%$
* $P(y=2) = 98\%$
* $P(y=3) = 0\%$
<br>
Logo, de acordo com o modelo, a instância em questão tem 98% de probabilidade de ser da classe 2, sendo classificada como tal.

# Exercício 2
A função de ativação ReLU é dada pela fórmula
<br>
$$ y = \max(0, x) $$
<br>
e possui comportamento constante em zero para todos os valores menores que zero e linear crescente para qualquer valor maior que zero. Essa função é a mais comumente utilizada em redes neurais convolutivas pois, além de ser rápida de calcular, ela não satura (não converge para um valor fixo de acordo com o crescimento de $y$, diferente da ativação sigmóide) e é ativada de forma esparsa, ou seja, a maioria das entradas $y$ serão transformadas em zero (não ativam o neurônio), simulando de forma mais realista a forma como o cérebro biológico, principal insipiração para redes neurais, funciona.

# Exercício 3 

In [14]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.utils import to_categorical

In [2]:
# Carrega o dataset
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

In [7]:
# Ajusta o formato dos dados para serem compativeis com o modelo de rede neural
# X_train: formato antigo = (60000, 28, 28) --> formato novo: (60000, 28, 28, 1)
# X_test: formato antigo = (10000, 28, 28) --> formato novo: (10000, 28, 28, 1)

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_test  = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)

In [15]:
# Ajusta o formato dos targets para serem compativeis com o modelo de rede neural
# Faz one-hot encoding

y_train = to_categorical(y_train)
y_test  = to_categorical(y_test)

### Modelo 1
* **Primeira camada:** convolução com 32 filtros 3x3 e ativação relu + max_pooling 2x2

* **Segunda camada:** convolução com 64 filtros 3x3 e ativação relu + max_pooling 2x2

* **Última camada:** *fully connected* com 10 neurônios (número de clases) e ativação softmax

In [19]:
model = Sequential()

# Primeira camada de convolucao e max-pooling
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=X_train.shape[1:]))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Segunda camdada de convolicao e max-pooling
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# "Achata" os dados para um vetor unidimensional
model.add(Flatten())

# Ultima camada: fully connected com 10 neuronios (1 para cada classe) e
# ativacao softmax para obter as probabilidades
model.add(Dense(10, activation='softmax'))

In [21]:
# Compila o modelo
# adam optimizer: otimizador que ajusta o learning rate
# categorical_crossentropy: usada para problemas de classificacoes com multiplas classes
# accuracy: metrica usada para ver a eficacia do modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [24]:
%%time
# Treina o modelo
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3)

Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 29s 486us/sample - loss: 0.0740 - accuracy: 0.9776 - val_loss: 0.0703 - val_accuracy: 0.9774
Epoch 2/3
60000/60000 [==============================] - 34s 567us/sample - loss: 0.0576 - accuracy: 0.9833 - val_loss: 0.0724 - val_accuracy: 0.9758
Epoch 3/3
60000/60000 [==============================] - 33s 544us/sample - loss: 0.0492 - accuracy: 0.9847 - val_loss: 0.0549 - val_accuracy: 0.9840
CPU times: user 3min 43s, sys: 2min 12s, total: 5min 55s
Wall time: 1min 35s


In [27]:
loss, acc = model.evaluate(X_train, y_train)

60000/60000 [==============================] - 10s 162us/sample - loss: 0.0338 - accuracy: 0.9897


In [28]:
print("Loss = ", loss, "\naccuracy = ", acc)

Loss =  0.03379081192867234 
accuracy =  0.98968333
